In [ ]:
import torch
import numpy as np
import cv2
from model_new import Encoder
from PIL import Image
import h5py
from torchvision import transforms

In [ ]:
cfg_large = {
        'input_channels':3,
        'patch_feature_channels':512,
        'global_feature_channels':256,
        'img_size':1024,
        'window_size':16,
        'embed_dim':192,
        'depth':[2,2,18],
        'num_heads':[6, 12, 24],
        'drop_path_rate':.2,
        'pretrain_window_size':[12, 12, 12],
        'unfreeze_backbone_modules':['head','norm','layers.2.blocks.14','layers.2.blocks.15','layers.2.blocks.16','layers.2.blocks.17']
    }

encoder_path = ''
encoder = Encoder(cfg_large)
encoder = encoder.eval().cuda()
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                ])
encoder.load_state_dict({k.replace("module.",""):v for k,v in torch.load(encoder_path).items()},strict=True)

In [ ]:
dataset_path = ''
data_idx = 0

def get_color(score,div):
    if score < div:
        return (255,0,0)
    else:
        return (0,255,0)

with h5py.File(dataset_path,'r') as f:
    keys = list(f.keys())
    img_raw = f[keys[data_idx]]['image_1'][:][988:2012,988:2012]

img = np.stack([img_raw] * 3,axis=-1)
img = transform(img)[None]

feat,conf = encoder(img)
conf = conf.squeeze()

img_output = img_raw
for line in range(conf.shape[0]):
    for samp in range(conf.shape[1]):
        cv2.circle(img_output,((samp + .5) * 16.,(line + .5) * 16),1,get_color(conf[line,samp],.5),-1)

Image.fromarray(img_output).show()



In [3]:
1500 + 512

2012